## lab-sql-python-connection

In [1]:
import mysql.connector
from getpass import getpass
import pandas as pd

#### Establish a Connection to Sakila Database

In [3]:
password = getpass()
cnx = mysql.connector.connect(user = "root",
                             password = password,
                             host = "localhost",
                             database = "sakila")
cnx.is_connected()

········


True

In [4]:
cursor = cnx.cursor()

In [28]:
month = "07"
year = 2005
query = (f"""SELECT * FROM sakila.rental
    WHERE
        YEAR(rental.rental_date) = {year}
    AND
        MONTH(rental.rental_date) = {month};""")
cursor.execute(query)
results = cursor.fetchall()
results

[(3470,
  datetime.datetime(2005, 7, 5, 22, 49, 24),
  883,
  565,
  datetime.datetime(2005, 7, 7, 19, 36, 24),
  1,
  datetime.datetime(2006, 2, 15, 21, 30, 53)),
 (3471,
  datetime.datetime(2005, 7, 5, 22, 51, 44),
  1724,
  242,
  datetime.datetime(2005, 7, 13, 1, 38, 44),
  2,
  datetime.datetime(2006, 2, 15, 21, 30, 53)),
 (3472,
  datetime.datetime(2005, 7, 5, 22, 56, 33),
  841,
  37,
  datetime.datetime(2005, 7, 13, 17, 18, 33),
  2,
  datetime.datetime(2006, 2, 15, 21, 30, 53)),
 (3473,
  datetime.datetime(2005, 7, 5, 22, 57, 34),
  2735,
  60,
  datetime.datetime(2005, 7, 12, 23, 53, 34),
  1,
  datetime.datetime(2006, 2, 15, 21, 30, 53)),
 (3474,
  datetime.datetime(2005, 7, 5, 22, 59, 53),
  97,
  594,
  datetime.datetime(2005, 7, 8, 20, 32, 53),
  1,
  datetime.datetime(2006, 2, 15, 21, 30, 53)),
 (3475,
  datetime.datetime(2005, 7, 5, 23, 1, 21),
  2189,
  8,
  datetime.datetime(2005, 7, 13, 23, 7, 21),
  2,
  datetime.datetime(2006, 2, 15, 21, 30, 53)),
 (3476,
  datetim

### Task 2

In [50]:
def rentals_month(month, year):
    if len(str(month))<2:
        month = f"0{month}"
    query = (f"""SELECT * FROM sakila.rental
    WHERE
        YEAR(rental.rental_date) = {year}
    AND
        MONTH(rental.rental_date) = {month};""")
    cursor.execute(query)
    results = cursor.fetchall()
    columns = [header[0] for header in cursor.description]
    df = pd.DataFrame(results, columns = columns)
    return df, month, year

In [44]:
df = rentals_month(0o7, 2005)
df = df.groupby("customer_id")["rental_id"].agg("count").reset_index()
df.columns = ["customer_id", "num_of_rentals"]
df

,customer_id,num_of_rentals
0,1,12
1,2,14
2,3,13
3,4,5
4,5,16
...,...,...
594,595,19
595,596,6
596,597,7
597,598,16


### Task 3

In [52]:
def rental_count_month(df, month, year):
    df = df.groupby("customer_id")["rental_id"].agg("count").reset_index()
    df.columns = ["customer_id", f"rentals_{month}_{year}"]
    return df

### Task 4

In [98]:
def compare_rentals():
    dfs = []
    for i in range(2):
        a,b,c = rentals_month(int(input("please enter a month: "),8), int(input("please enter a year: ")))
        dfs.append(rental_count_month(a, b, c))
        
    new_df = pd.merge(left=dfs[0], right=dfs[1], on="customer_id")
    new_df["difference"] = new_df[new_df.columns[1]] - new_df[new_df.columns[2]]
    return new_df

In [99]:
compare_rentals()

please enter a month: 6
please enter a year: 2005
please enter a month: 5
please enter a year: 2005


,customer_id,rentals_06_2005,rentals_05_2005,difference
0,1,7,2,5
1,2,1,1,0
2,3,4,2,2
3,5,5,3,2
4,6,4,3,1
...,...,...,...,...
507,594,6,4,2
508,595,2,1,1
509,596,2,6,-4
510,597,3,2,1
